In [1]:
import json
import copy

In [2]:
# Pulling in data about Fantasy League owners and MLB teams
owners = json.load(open('data/owners.json'))
mlb_teams = json.load(open('data/mlb-teams.json'))

In [3]:
# Create a new file that will hold all the league owner's preferences for other teams, with a default weight of 0
mlb_teams_weights = [{"teamId": team['teamId'], "teamName": team['fullName'], "ownerPreferenceWeight": 0} for team in mlb_teams]
team_weights = [{**owner, "mlbWeights": mlb_teams_weights} for owner in owners]

### Establishing MLB team biases
For now, we're going to manually identify who are some of the best and worst teams in the MLB and manually set a bias for those teams that will apply to each Fantasy owner's preference for players on that MLB team.

This could be refactored in a future version to be dynamic based on last year's standings, projected power rankings, or even custom projections for each team.

In [4]:
# Declaring our default biases
default_bias = [
  {"teamId": 18, "ownerPreferenceWeight": 0.1},
  {"teamId": 19, "ownerPreferenceWeight": 0.1},
  {"teamId": 15, "ownerPreferenceWeight": 0.08},
  {"teamId": 16, "ownerPreferenceWeight": 0.08},
  {"teamId": 21, "ownerPreferenceWeight": 0.06},
  {"teamId": 22, "ownerPreferenceWeight": 0.06},
  {"teamId": 12, "ownerPreferenceWeight": 0.04},
  {"teamId": 14, "ownerPreferenceWeight": 0.04},
  {"teamId": 25, "ownerPreferenceWeight": 0.04},
  {"teamId": 26, "ownerPreferenceWeight": 0.04},
  {"teamId": 2, "ownerPreferenceWeight": -0.02},
  {"teamId": 29, "ownerPreferenceWeight": -0.02},
  {"teamId": 17, "ownerPreferenceWeight": -0.04},
  {"teamId": 28, "ownerPreferenceWeight": -0.04},
  {"teamId": 6, "ownerPreferenceWeight": -0.06},
  {"teamId": 27, "ownerPreferenceWeight": -0.06},
  {"teamId": 20, "ownerPreferenceWeight": -0.08},
  {"teamId": 23, "ownerPreferenceWeight": -0.08},
  {"teamId": 11, "ownerPreferenceWeight": -0.1},
]

# I'm going to double these biases because I want make sure they're effective on a 0-1 scale
for bias in default_bias:
  bias['ownerPreferenceWeight'] *= 2

In [19]:
# Let's update our biases to our original mlb_teams_weights, then rewrite the team_weights file
for weight in mlb_teams_weights:
  for bias in default_bias:
    if weight['teamId'] == bias['teamId']:
      weight['ownerPreferenceWeight'] = bias['ownerPreferenceWeight']

baseline_team_weights = [{**owner, "mlbWeights": mlb_teams_weights} for owner in owners]

In [ ]:
# Now we will export this data as a base file before we start refining further
with open('outputs/owner-mlb-team-weights.json', 'w') as outfile:
  json.dump(baseline_team_weights, outfile)

In [7]:
# Instead, I have again manually set personal biases for each owner, which we will
# import from another file
owner_biases = json.load(open('data/owner-bias-input.json'))

In [20]:


# We will update our team_weights file with the biases defined in owner_biases using the ownerIds and teamIds
new_file = []

# Iterates through our base file that has all of the our current owner's MLB team preference weights
for owner in baseline_team_weights:
  new_object = {}
  id1 = str(owner['teamId'])

  # Iterates through our new biases file that has all of the owner's refined personal biases
  for bias in owner_biases:
    id2 = str(bias['ownerTeamId'])

    # If the owner's Fantasy teamId matches the owner's Fantasy teamId in the biases file, we will proceed
    if id1 == id2:
      # We will iterate through the Fantasy team owner's refined biases from the biases input file and make a decision on how to update the base file
      for weight in bias['mlbWeights']:
        old_weight = owner['mlbWeights'][weight['teamId']-1]['ownerPreferenceWeight']
        new_weight = weight['ownerPreferenceWeight']
        log = {
          "ownerTeamId": owner['teamId'],
          "ownerName": owner['ownerName'],
          "adjustingWeightOf": str(weight['teamId']) + " - " + owner['mlbWeights'][weight['teamId']-1]['teamName'],
          "baselineWeight": old_weight,
          "adjustedWeightToBeAdded": new_weight,
        }
        print(log)

        new_object = {
          "ownerId": owner['ownerId'],
          "ownerName": owner['ownerName'],
          "teamId": owner['teamId'],
          "mlbWeights": owner['mlbWeights']
        }

        # If the new weight is greater than 1, we will set it to 1
        if old_weight + new_weight > 1:
          new_object['mlbWeights'][weight['teamId']-1]['ownerPreferenceWeight'] = 1
        # If the new weight is less than -1, we will set it to -1
        elif old_weight + new_weight < -1:
          new_object['mlbWeights'][weight['teamId']-1]['ownerPreferenceWeight'] = -1
        # Otherwise, we will just add the new weight to the old weight
        else:
          new_object['mlbWeights'][weight['teamId']-1]['ownerPreferenceWeight'] = old_weight + new_weight
        
        new_file.append(new_object)
        print("confirming the weight is now:", new_object['mlbWeights'][weight['teamId']-1]['ownerPreferenceWeight'], "for teamId:", weight['teamId'])
    
    else:
      print("No match", id1, id2)
      continue
    
    



{'ownerTeamId': 1, 'ownerName': 'Zack Bessette', 'adjustingWeightOf': '25 - San Diego Padres', 'baselineWeight': 0.4, 'adjustedWeightToBeAdded': 1}
confirming the weight is now: 1 for teamId: 25
{'ownerTeamId': 1, 'ownerName': 'Zack Bessette', 'adjustingWeightOf': '18 - Houston Astros', 'baselineWeight': 0.4, 'adjustedWeightToBeAdded': 0.2}
confirming the weight is now: 0.6000000000000001 for teamId: 18
{'ownerTeamId': 1, 'ownerName': 'Zack Bessette', 'adjustingWeightOf': '21 - New York Mets', 'baselineWeight': 0.4, 'adjustedWeightToBeAdded': 0.1}
confirming the weight is now: 0.5 for teamId: 21
{'ownerTeamId': 1, 'ownerName': 'Zack Bessette', 'adjustingWeightOf': '17 - Cincinnati Reds', 'baselineWeight': 0.4, 'adjustedWeightToBeAdded': 0.1}
confirming the weight is now: 0.5 for teamId: 17
{'ownerTeamId': 1, 'ownerName': 'Zack Bessette', 'adjustingWeightOf': '22 - Philadelphia Phillies', 'baselineWeight': 0.4, 'adjustedWeightToBeAdded': 0.1}
confirming the weight is now: 0.5 for teamId

In [ ]:
# Now we will export this data as a final file
with open('outputs/owner-mlb-team-weights-adjusted.json', 'w') as outfile:
  json.dump(team_weights, outfile)